In [ ]:
!pip install git+git://github.com/GerbenBeintema/gym-unbalanced-disk@master

In [2]:
import gym, time
import gym_unbalanced_disk #is required
import numpy as np
env = gym.make('unbalanced-disk-v0',dt=0.05,umax=4) #if this fails restart the kernel or use gym_unbalanced_disk.UnbalancedDisk
env_sin = gym.make('unbalanced-disk-sincos-v0',dt=0.05,umax=4) # angle expressed in sine and cos components. (useful for RL)


# First test

Look at the rewards

In [4]:
Umax = 4

T = 20 #c)
ulist = np.sign(np.sin(np.arange(200)/T*2*np.pi))*Umax #c=)

obs = env.reset()
# obs_s = env_sin.reset()
max_reward = 0
min_reward = 0.0001
try:
    for u in ulist:
        obs, reward, done, info = env.step(u)
        # Check rewards
        if reward>max_reward: max_reward=reward
        if reward<min_reward: min_reward=reward
#         obs_s, reward_s, done, info = env_sin.step(u)
        print(obs,reward)
#         print(obs_s,reward_s)
        env.render()
        time.sleep(1/24)
        if done:
            obs = env.reset()
#             obs_s  = env_sin.reset()
finally: #this will always run
    env.close()
#     env_sin.close()

[0.0007108280808274742, -0.004109756378511632] 0.999999803052445
[0.12389276532644575, 4.702050820056832] 0.9628511762639276
[0.43860521483735365, 7.5279414661437904] 0.6201520288114745
[0.8370085239698266, 8.099109328904078] 0.17487759106192513
[1.222498965876134, 7.053360870827298] 0.02463019174334393
[1.5342760329604048, 5.402357940523033] 0.002914979444146621
[1.7642958994987186, 3.8618347273583327] 0.00044253926229424854
[1.927932870526863, 2.705020047508972] 0.00010002419418342004
[2.0427215436274535, 1.9544711443345193] 3.2279709196350335e-05
[2.12648243170647, 1.5555807575846026] 1.3202914203501245e-05
[2.2009271220964473, 1.446624375950776] 5.968482904834381e-06
[2.0200866879327073, -8.6168295942562] 4.016775904190912e-05
[1.341807856085334, -18.290710359189706] 0.011294693109513034
[0.23485907353951477, -25.137735925240094] 0.8718980466436035
[-1.0440196855369295, -24.8884241322219] 0.06642828615713363
[-2.1971627728089547, -21.346281405380164] 6.23234563916665e-06
[-3.246036

[-6.152116510949531, 2.0491245196842955] 0.9581808559980154
[-6.204490256580683, -3.919342235828944] 0.984252943020583
[-6.511983007924133, -8.061661558730936] 0.87707547276787
[-6.96362926344491, -9.506732263262627] 0.3168535340319577
[-7.425461630906095, -8.6952146220037] 0.039179189459125886
[-7.817956105448818, -6.910191863420924] 0.002908638843834357
[-8.118516962171933, -5.221399211484362] 0.0002337806955774274
[-8.092341071765278, 5.998004082290862] 0.00029331214581287604
[-7.53703885994503, 16.074795674104127] 0.02030969981531435
[-6.534703187410877, 23.08339462393862] 0.8543935556628809
[-5.345228766643432, 23.38795239573935] 0.11226620910803836
[-4.260503625084687, 19.984560537440736] 3.8614991950552026e-05
[-3.298373393540603, 19.265818995956536] 2.499547396147341e-10
[-2.240001213483367, 23.95447742378501] 3.858124304079077e-06
[-0.849641115243442, 31.523376629433542] 0.16661182313623235
[0.7994791227451843, 32.8287616125298] 0.2039194383507122
[2.3293320915379474, 28.57467

In [5]:
print(max_reward)
print(min_reward)

0.9999999293191881
2.8815379246051137e-11


# Q-learning

In [17]:
from gym import Wrapper
from gym import spaces
from collections import deque #a list with a fixed lenght
class histwrapper(Wrapper):
    def __init__(self, env, Nhist=7):
        super(histwrapper, self).__init__(env) #sets self.env
        
        shape = env.observation_space.shape #load the obs shape
        shapehist = (Nhist,) + shape #add (Nhist)
        h = env.observation_space.high*np.ones(shapehist) #the upperbounds
        l = env.observation_space.low*np.ones(shapehist) #the lowerbounds
        self.observation_space = spaces.Box(low=l.astype(np.float32),high=h.astype(np.float32)) #modify the observation_space
        self.hist = deque([],Nhist)
        self.Nhist = Nhist
        
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.hist.append(observation)
        observation = np.array(self.hist)
        return observation, reward, done, info

    def reset(self):
        observation = self.env.reset()
        self.hist.extend([observation]*self.Nhist)
        observation = np.array(self.hist)
        return observation
env = gym_unbalanced_disk.UnbalancedDisk(dt=0.025, umax=4.)

env = histwrapper(env, Nhist=4)
# env.reset()
# print(env.step(0))
# print(env.step(0))### Discretizer

### Discretizer

Since the observations are continuous we have to discretize these observations

In [18]:
class Discretize_obs(gym.Wrapper):
    def __init__(self, env, nvec=10):
        super(Discretize_obs, self).__init__(env) #sets self.env
        if isinstance(nvec,int): #nvec in each dimention
            self.nvec = [nvec]*np.prod(env.observation_space.shape,dtype=int)
        else:
            self.nvec = nvec
        self.nvec = np.array(nvec) #(Nobs,) array
        
        self.observation_space = gym.spaces.MultiDiscrete(self.nvec) 
        self.olow, self.ohigh = env.observation_space.low, env.observation_space.high 
        self.olow[0]=-1e12
        self.ohigh[0]=1e12
    def discretize(self,observation): 
        return tuple(((observation - self.olow)/(self.ohigh - self.olow+1e-12)*self.nvec).astype(int)) 
        
    def step(self, action):
        observation, reward, done, info = self.env.step(action) 
        return self.discretize(observation), reward, done, info 

    def reset(self):
        return self.discretize(self.env.reset()) 

env = gym_unbalanced_disk.UnbalancedDisk(dt=0.025, umax=4.)
env = Discretize_obs(env,nvec=100)
env.reset()
print(env.step(0)) #testing
print(env.step(0))
print(env.step(0))
print(env.step(0))
print(env.step(2))
print(env.step(2))
print(env.step(2))

[inf 40.]
((50, 49), 0.9999987655122206, False, {})
((49, 49), 0.9999991284427118, False, {})
((49, 49), 0.999999515155028, False, {})
((50, 49), 0.9999998228803474, False, {})
((50, 51), 0.999353537709095, False, {})
((50, 52), 0.9906222102273687, False, {})
((50, 53), 0.9580218275155943, False, {})


In [19]:
def argmax(a):
    #random argmax
    a = np.array(a)
    return np.random.choice(np.arange(len(a),dtype=int)[a==np.max(a)])

def roll_mean(ar,start=2000,N=50): #smoothing if needed
    s = 1-1/N
    k = start
    out = np.zeros(ar.shape)
    for i,a in enumerate(ar):
        k = s*k + (1-s)*a
        out[i] = k
    return out

def Qlearn(env, nsteps=5000, callbackfeq=100, alpha=0.2,eps=0.2, gamma=0.99):
    env.action_space.n = 3
    from collections import defaultdict
    Qmat = defaultdict(float) #any new argument set to zero
    env_time = env
    while not isinstance(env_time,gym.wrappers.time_limit.TimeLimit):
        env_time = env_time.env
    ep_lengths = []
    ep_lengths_steps = []
    
    obs = env.reset()
    print('goal reached time:')
    for z in range(nsteps):

        if np.random.uniform()<eps:
            action = env.action_space.sample()
        else:
            action = argmax([Qmat[obs,i] for i in range(env.action_space.n)])

        obs_new, reward, done, info = env.step(action)

        if done and not info.get('TimeLimit.truncated', False): #terminal state = done and not by timeout
            #saving results:
            print(env_time._elapsed_steps, end=' ')
            ep_lengths.append(env_time._elapsed_steps)
            ep_lengths_steps.append(z)
            
            #updating Qmat:
            A = reward - Qmat[obs,action] # adventage or TD
            Qmat[obs,action] += alpha*A
            obs = env.reset()
        else: #done by timeout or not done
            A = reward + gamma*max(Qmat[obs_new, - Qmat[obs,action]
            Qmat[obs,action] += alpha*A
            obs = obs_new
            
            if info.get('TimeLimit.truncated',False): #done by timeout
                #saving results:
                ep_lengths.append(env_time._elapsed_steps)
                ep_lengths_steps.append(z)
                print('out', end=' ')
                
                #reset:
                obs = env.reset()
    print()
    
    return Qmat, np.array(ep_lengths_steps), np.array(ep_lengths)



for nvec in [5,10,20]: 
    max_episode_steps = 1000
    env = gym_unbalanced_disk.UnbalancedDisk(dt=0.025, umax=4.) 
    env = gym.wrappers.time_limit.TimeLimit(env,max_episode_steps=max_episode_steps) 
    env = Discretize_obs(env,nvec=nvec) 

    print('nvec=',nvec)
    Qmat, ep_lengths_steps, ep_lengths = Qlearn(env, nsteps=400_000, callbackfeq=5000) 
    plt.plot(ep_lengths_steps,roll_mean(ep_lengths,start=max_episode_steps),label=str(nvec)) 
plt.legend()
plt.show() 


nvec= 5
goal reached time:


TypeError: unhashable type: 'numpy.ndarray'